In [66]:
# %pip install -e ../..

Obtaining file:///Users/sethvanderbijl/Coding%20Projects/DM_project_2
  Attempting uninstall: DM-project-2
    Found existing installation: DM-project-2 1.0
    Uninstalling DM-project-2-1.0:
      Successfully uninstalled DM-project-2-1.0
  Running setup.py develop for DM-project-2
You should consider upgrading via the '/Users/sethvanderbijl/Coding Projects/DM_project_2/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from data.load import load_data
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, RandomForestRegressor, StackingClassifier, StackingRegressor
from sklearn.linear_model import BayesianRidge, ElasticNet, HuberRegressor, LinearRegression, SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import std
import time

import matplotlib.pyplot as plt
import copy
from helpers.ranking import to_ranking


/Users/sethvanderbijl/Coding Projects/DM_project_2/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Load data
svo = [('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg'), ('prop_id', 'price_usd', 'avg'), ('srch_destination_id', 'price_usd', 'avg'), ('site_id', 'prop_starrating', 'avg'), ('srch_id', 'prop_starrating', 'avg'), ('prop_id', 'prop_starrating', 'avg'), ('srch_destination_id', 'prop_starrating', 'avg'), ('site_id', 'prop_review_score', 'avg'), ('srch_id', 'prop_review_score', 'avg'), ('prop_id', 'prop_review_score', 'avg'), ('srch_destination_id', 'prop_review_score', 'avg'), ('site_id', 'prop_log_historical_price', 'avg'), ('srch_id', 'prop_log_historical_price', 'avg'), ('prop_id', 'prop_log_historical_price', 'avg'), ('srch_destination_id', 'prop_log_historical_price', 'avg'),
('srch_destination_id', 'prop_location_score1', 'avg'),('prop_id', 'prop_location_score1', 'avg'),('srch_id', 'prop_location_score1', 'avg'),('site_id', 'prop_location_score1', 'avg')]

fts=[('prop_starrating', 'visitor_hist_starrating', 'diff'), ('same_val_prop_id_price_usd_avg', 'same_val_prop_id_prop_log_historical_price_avg', 'all'), ('same_val_prop_id_price_usd_avg', 'prop_review_score', 'all'), ('same_val_srch_destination_id_price_usd_avg', 'same_val_srch_destination_id_prop_log_historical_price_avg', 'all'), ('same_val_srch_destination_id_price_usd_avg', 'prop_review_score', 'all'), ('prop_location_score2', 'prop_location_score1', 'all')]

order = [('srch_id', 'all'), ('prop_id', 'price_usd')]

# This gives 0.524 at 50's!

df = load_data(add_day_parts=True, same_value_operations=svo, fts_operations=fts, add_seasons=True, add_dist_holidays=True, scaling=True, order_operations=order, caching=True)
# df = pd.read_csv('for_colab.csv')

No cache for this specific request, start loading base df from disk.
Using cache: /Users/sethvanderbijl/Coding Projects/DM_project_2/data/caches/dayparts('train', None).h5
Using cache: /Users/sethvanderbijl/Coding Projects/DM_project_2/data/caches/seasons('train', None).h5
Using cache: /Users/sethvanderbijl/Coding Projects/DM_project_2/data/caches/dist_holiday('train', None).h5
generating same id features


100%|██████████| 20/20 [09:13<00:00, 27.67s/it]


generating order features


100%|██████████| 2/2 [11:18<00:00, 339.18s/it]


Feature engineering columns with operations
saving result of specific arguments to cache


100%|██████████| 500/500 [04:43<00:00,  1.76it/s]  


In [3]:
df.head(50).to_csv('check.csv')

In [4]:


# Split into target and predictors
y = df['booking_bool']*5 + df['click_bool']
X = df.drop(['booking_bool','click_bool', 'position', 'gross_bookings_usd', 'date_time'], axis=1)
X = X.fillna(X.mean())
del df

In [5]:
X_train = X[:4900000]
X_eval = X[4900000:]
y_train = y[:4900000]
y_eval = y[4900000:]

qids_train = X_train.groupby('srch_id')['srch_id'].count().to_numpy()
qids_eval = X_eval.groupby('srch_id')['srch_id'].count().to_numpy()


In [8]:
# Set up stakcing regressor
# rf = RandomForestRegressor()
# rf= StackingRegressor([('rf',RandomForestRegressor()), ('br',BayesianRidge()), ('ab',AdaBoostRegressor())], verbose=3, n_jobs=7)
# rf=RandomForestRegressor(verbose=3)
from datetime import datetime
import lightgbm
model = lightgbm.LGBMRanker(objective='lambdarank', n_estimators=400, metric='ndcg', n_jobs=4, num_leaves=60)
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
   
    eval_set=[(X_eval, y_eval)],
    eval_group=[qids_eval],
    eval_at=[5, 10],
    early_stopping_rounds=200,
)


/Users/sethvanderbijl/Coding Projects/DM_project_2/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's ndcg@5: 0.291663	valid_0's ndcg@10: 0.36544	valid_0's ndcg@20: 0.420864	valid_0's ndcg@50: 0.443126	valid_0's ndcg@100: 0.443126	valid_0's ndcg@150: 0.443126	valid_0's ndcg@200: 0.443126
[2]	valid_0's ndcg@5: 0.324674	valid_0's ndcg@10: 0.39688	valid_0's ndcg@20: 0.448782	valid_0's ndcg@50: 0.466921	valid_0's ndcg@100: 0.466921	valid_0's ndcg@150: 0.466921	valid_0's ndcg@200: 0.466921
[3]	valid_0's ndcg@5: 0.346149	valid_0's ndcg@10: 0.411682	valid_0's ndcg@20: 0.462168	valid_0's ndcg@50: 0.479153	valid_0's ndcg@100: 0.479153	valid_0's ndcg@150: 0.479153	valid_0's ndcg@200: 0.479153
[4]	valid_0's ndcg@5: 0.352626	valid_0's ndcg@10: 0.419056	valid_0's ndcg@20: 0.466245	valid_0's ndcg@50: 0.483208	valid_0's ndcg@100: 0.483208	valid_0's ndcg@150: 0.483208	valid_0's ndcg@200: 0.483208
[5]	valid_0's ndcg@5: 0.361086	valid_0's ndcg@10: 0.42571	valid_0's ndcg@20: 0.472855	valid_0's ndcg@50: 0.489144	valid_0's ndcg@100: 0.489144	valid_0's ndcg@150: 0.489144	valid_0's ndcg@200: 0.

LGBMRanker(metric='ndcg', n_estimators=400, n_jobs=4, num_leaves=60,
           objective='lambdarank')

In [7]:
# Feature importances
t=zip(model.feature_name_, model.feature_importances_)
t = pd.DataFrame(t, columns=['n','i']).sort_values('i', ascending= False)
print(t.to_numpy())

[['price_usd' 584]
 ['order_srch_id_price_usd' 501]
 ['order_prop_id_price_usd' 385]
 ['prop_location_score2' 364]
 ['prop_log_historical_price' 275]
 ['prop_location_score2_diff_prop_location_score1' 271]
 ['prop_location_score2_div_prop_location_score1' 263]
 ['random_bool' 234]
 ['prop_starrating' 226]
 ['same_val_prop_id_prop_log_historical_price_avg' 220]
 ['order_srch_id_prop_location_score2' 200]
 ['prop_id' 196]
 ['same_val_srch_id_price_usd_avg' 188]
 ['order_srch_id_same_val_prop_id_price_usd_avg' 186]
 ['same_val_prop_id_price_usd_avg_mult_prop_review_score' 180]
 ['prop_review_score' 164]
 ['same_val_prop_id_price_usd_avg_div_same_val_prop_id_prop_log_historical_price_avg'
  162]
 ['same_val_prop_id_price_usd_avg_div_prop_review_score' 161]
 ['srch_query_affinity_score' 161]
 ['orig_destination_distance' 157]
 ['same_val_srch_id_prop_location_score1_avg' 156]
 ['srch_booking_window' 153]
 ['same_val_srch_destination_id_prop_location_score1_avg' 148]
 ['order_srch_id_same_va

In [9]:
model.best_score_

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('ndcg@5', 0.4079910467852049),
                          ('ndcg@10', 0.4675817077285257),
                          ('ndcg@20', 0.509955663893552),
                          ('ndcg@50', 0.5229719593468461)])})

In [ ]:
# Load our own test data
# test = load_data(mode='kaggle_test', add_day_parts=True, same_value_operations=[('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg'), ('prop_id', 'price_usd', 'avg'), ('srch_destination_id', 'price_usd', 'avg')], fts_operations=[('prop_starrating', 'visitor_hist_starrating', 'diff')], add_seasons=True, scaling=True)


No cache for this specific request, start loading base df from disk.


KeyboardInterrupt: 

In [ ]:
# # Drop irrelevant columns from our own test data
# test = test.drop(['date_time'], axis=1)
# X_test = test.fillna(X.mean()) #Mean of x or mean of x_test?
# del test

# results = model.predict(X_test)
# print(len(results.tolist()))

4959183


In [ ]:
# # Check if there is a difference in columns (there shouldn't be)
# list(set(X_test.columns) - set(X.columns))

[]

In [ ]:


# new_df = X_test[['prop_id', 'srch_id']]
# new_df['predictions'] = results
# print(new_df.head(30))


    prop_id  srch_id  predictions
0      3180        1    -0.912714
1      5543        1    -0.898955
2     14142        1    -0.481791
3     22393        1    -1.223566
4     24194        1    -0.534830
5     28181        1    -0.468284
6     34263        1    -2.020343
7     37567        1    -0.496871
8     50162        1    -0.030542
9     54937        1     0.026699
10    56050        1    -1.239528
11    61632        1    -1.168983
12    61934        1     0.009585
13    63894        1    -0.412825
14    72090        1    -0.466601
15    73666        1    -0.846502
16    74045        1    -0.508723
17    78599        1    -0.611964
18    82231        1    -0.385085
19    89466        1    -0.854499
20    90385        1    -0.416553
21    94729        1    -0.692877
22    95031        1    -0.521723
23    99484        1     0.316063
24   123675        1    -0.518262
25   128085        1    -0.412545
26   128871        1    -0.987940
27   134992        1    -1.286324
28   139162   

/var/folders/gx/c6cqbmz17cqcb4nq4ynrjcn80000gn/T/ipykernel_69518/1278978652.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['predictions'] = results


In [ ]:
# sorted_df = new_df.sort_values(['srch_id', 'predictions'], ascending=[True, False])
# print(sorted_df.head(30))

    prop_id  srch_id  predictions
23    99484        1     0.316063
9     54937        1     0.026699
12    61934        1     0.009585
8     50162        1    -0.030542
18    82231        1    -0.385085
25   128085        1    -0.412545
13    63894        1    -0.412825
20    90385        1    -0.416553
14    72090        1    -0.466601
5     28181        1    -0.468284
2     14142        1    -0.481791
7     37567        1    -0.496871
16    74045        1    -0.508723
24   123675        1    -0.518262
22    95031        1    -0.521723
4     24194        1    -0.534830
17    78599        1    -0.611964
21    94729        1    -0.692877
15    73666        1    -0.846502
19    89466        1    -0.854499
1      5543        1    -0.898955
0      3180        1    -0.912714
26   128871        1    -0.987940
11    61632        1    -1.168983
3     22393        1    -1.223566
10    56050        1    -1.239528
28   139162        1    -1.244546
27   134992        1    -1.286324
6     34263   

In [ ]:
# export_df = sorted_df.drop(['predictions'], axis=1)
# export_df = export_df[['srch_id', 'prop_id']] # swap columns for required format

# compression_opts = dict(method='zip', archive_name='out.csv')  
# export_df.to_csv('out.zip', index=False, compression=compression_opts) # we can submit .zip instead of the raw .csv, which saves some space.